In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import fasttext
import fasttext_models as fm

In [2]:
print("First thing's first, we need to make our models.\nWe will be building 10 models in total: 5 will be trained on cleaned and lemmatized data, 5 will be trained on the raw text. Each of the 5 for those two training sets will vary by vector length: 25, 50, 100, 200, and 300.")
# execfile('fasttext_models.py') - Does not work in python 3, use:
#exec(open("./fasttext_models.py").read())

mcs,mus = ([mc0,mc1,mc2,mc3,mc4],[mu0,mu1,mu2,mu3,mu4]) = fm.main()

First thing's first, we need to make our models.
We will be building 10 models in total: 5 will be trained on cleaned and lemmatized data, 5 will be trained on the raw text. Each of the 5 for those two training sets will vary by vector length: 25, 50, 100, 200, and 300.
reviews_train.json loaded:
   Id  HelpfulnessNumerator  HelpfulnessDenominator  Score  \
0   1                     1                       1      5   
1   2                     0                       0      1   
2   3                     1                       1      4   
3   4                     3                       3      2   
4   5                     0                       0      5   

                 Summary                                               Text  \
0  Good Quality Dog Food  I have bought several of the Vitality canned d...   
1      Not as Advertised  Product arrived labeled as Jumbo Salted Peanut...   
2  "Delight" says it all  This is a confection that has been around a fe...   
3         Cou

In [3]:
# load the models for evaluation
'''
mc0 = fasttext.load_model('data/fasttext_skipgram_cleaned_D25.bin')
mc1 = fasttext.load_model('data/fasttext_skipgram_cleaned_D50.bin')
mc2 = fasttext.load_model('data/fasttext_skipgram_cleaned_D100.bin')
mc3 = fasttext.load_model('data/fasttext_skipgram_cleaned_D200.bin')
mc4 = fasttext.load_model('data/fasttext_skipgram_cleaned_D300.bin')
mcs = [mc0,mc1,mc2,mc3,mc4]

for m in mcs :
    print(m.get_dimension())
mu0 = fasttext.load_model('data/fasttext_skipgram_uncleaned_D25.bin')
mu1 = fasttext.load_model('data/fasttext_skipgram_uncleaned_D50.bin')
mu2 = fasttext.load_model('data/fasttext_skipgram_uncleaned_D100.bin')
mu3 = fasttext.load_model('data/fasttext_skipgram_uncleaned_D200.bin')
mu4 = fasttext.load_model('data/fasttext_skipgram_uncleaned_D300.bin')
mus = [mu0,mu1,mu2,mu3,mu4]
'''

print("\nHere are the top 50 words for the model trained on cleaned data with D100:")
mc2_words = mc4.get_words()
print(mc2_words[:50])
print("\nHere are the top 50 words for the model trained on uncleaned data with D100:")
mu2_words = mu2.get_words()
print(mu2_words[:50])

print("Nota Bene: I cannot figure out how to remove some of these symbols. I am pretty sure the strategy I used in fasttext_models.py works, but it is still not working out. My theory is that some of these are subwords... but I doubt that it is that simple.")


Here are the top 50 words for the model trained on cleaned data with D100:
['>', '<', 'br', '</s>', 'like', 'good', 'taste', 'flavor', 'one', 'get', 'love', 'product', 'make', 'use', 'coffee', 'great', 'try', 'well', 'food', 'buy', 'tea', 'find', 'would', 'eat', 'dog', 'go', 'really', 'time', 'much', 'amazon', 'order', 'also', 'price', 'bag', 'cup', 'give', 'little', 'even', 'drink', 'say', 'think', 'store', 'day', 'cat', 'add', 'box', 'chocolate', 'treat', 'come', 'first']

Here are the top 50 words for the model trained on uncleaned data with D100:
['the', 'I', 'and', 'a', 'to', 'of', 'is', 'it', '</s>', 'for', 'in', 'this', 'that', 'my', 'with', 'have', 'but', 'are', 'was', 'not', 'you', '/><br', 'on', 'as', 'like', 'they', 'so', 'be', 'The', 'or', 'at', 'these', 'just', 'them', 'very', 'from', 'one', 'good', 'It', '"I', 'has', 'can', 'taste', 'will', 'would', 'had', 'all', 'more', 'than', 'when']
Nota Bene: I cannot figure out how to remove some of these symbols. I am pretty sure 

In [43]:
import get_accuracy

print("Now let's compare our 10 different models' accuracy, precision, and recall.")
print("First, a quick note about precision and recall: The fasttext test() function calculates  precision and recall \"at k\" for our models. Note that these are not the usual definitions of precision and recall used in most discussion about binary classifiers. More precisely, precision at k is:\n      P@k = r / k,\n      the # of relevant labels r divided by the number of top predictions k.")
print("In our case, P@1 will be either 1 or 0 in each test case, because the top guess either is or is not the correct label.\nFor our purposes, just considering precision and recall at 1 makes sense, because there is only one label we care about: the correct one. When k>1, our models can only get a value of either 1/k or 0 in each test case, making this a poor measure of accuracy.\nIt is not explicitly stated in the fasttext documentation, but it is reasonable to assume that P@1 will give the same output as we would get using the sci kit library's tools.\nSpecifically, it gives the same results as sklearn.metrics.precision_score(average='micro') would. This means that fasttext.test() is calculating metrics globally by counting the total true positives, false negatives and false positives across all the labels and producing one number for the precision and recall score. Unfortunately, there seems to be no way to change this calculation beyond varying the k value.")
print("To calculate accuracy for comparision with our other models (with different embeddings), we need to use our own code. See the module get_accuracy.py for more informaiton.\n\n\tThe following dataframe compares our models' preformance on test data:")

cdata = []
for m in mcs :
    data = []
    data.append('Clean')
    data.append(m.get_dimension())
    n,p,r = m.test('data/reviews_uncleaned.test', k=1)
    data.append(get_accuracy.get_accuracy(m))
    data.append(p)
    data.append(r)
    data.append(n)
    cdata.append(data)
df1 = pd.DataFrame(cdata,columns = ['Train Data','Word Vector Size','Accuracy','Precision @1','Recall @1','N'])

udata = []
for m in mus :
    data = []
    data.append('Unclean')
    data.append(m.get_dimension())
    n,p,r = m.test('data/reviews_uncleaned.test', k=1)
    data.append(get_accuracy.get_accuracy(m))
    data.append(p)
    data.append(r)
    data.append(n)
    udata.append(data)
df2 = pd.DataFrame(udata,columns = ['Train Data','Word Vector Size','Accuracy','Precision @1','Recall @1','N'])

df_merged=pd.concat([df1,df2], ignore_index=True)
df_merged

Now let's compare our 10 different models' accuracy, precision, and recall.
First, a quick note about precision and recall: The fasttext test() function calculates  precision and recall "at k" for our models. Note that these are not the usual definitions of precision and recall used in most discussion about binary classifiers. More precisely, precision at k is:
      P@k = r / k,
      the # of relevant labels r divided by the number of top predictions k.
In our case, P@1 will be either 1 or 0 in each test case, because the top guess either is or is not the correct label.
For our purposes, just considering precision and recall at 1 makes sense, because there is only one label we care about: the correct one. When k>1, our models can only get a value of either 1/k or 0 in each test case, making this a poor measure of accuracy.
It is not explicitly stated in the fasttext documentation, but it is reasonable to assume that P@1 will give the same output as we would get using the sci kit libr

,Train Data,Word Vector Size,Accuracy,Precision @1,Recall @1,N
0,Clean,25,0.588402,0.588402,0.588402,113691
1,Clean,50,0.590777,0.590777,0.590777,113691
2,Clean,100,0.590407,0.590407,0.590407,113691
3,Clean,200,0.587892,0.587892,0.587892,113691
4,Clean,300,0.590680,0.590680,0.590680,113691
5,Unclean,25,0.686396,0.686396,0.686396,113691
6,Unclean,50,0.686774,0.686774,0.686774,113691
7,Unclean,100,0.687038,0.687038,0.687038,113691
8,Unclean,200,0.686774,0.686774,0.686774,113691
9,Unclean,300,0.686906,0.686906,0.686906,113691


In [5]:
print("Now let's take a look at some of the vector embeddings.")
print("Let's look at the vector for \"chocolate\" in the cleaned-data model:\n")
print(mc2.get_word_vector('chocolate'))
print("\nWe can also look at the most similar words, or the vector's 'neighbors.' These are determined by the cosine similarity of vectors.")
print(mc2.get_nearest_neighbors('chocolate'))

print("\nFasttext also lets us try out analogies. Let's see how it does with the following:")
print("\n\"\'hot\' is to \'cold\', what \'good\' is to _____\"")
print(mc2.get_analogies('hot','cold','good'))

Now let's take a look at some of the vector embeddings.
Let's look at the vector for "chocolate" in the cleaned-data model:

[-9.51359496e-02 -4.76726629e-02  1.50278946e-02  6.76411688e-02
 -2.81996746e-02 -2.65564267e-02  3.68301980e-02  1.05108298e-01
  2.63368885e-04 -6.05497696e-02 -7.75734782e-02 -3.38424481e-02
  3.83751877e-02 -2.14667842e-02  1.07903354e-01 -8.65911245e-02
 -8.62116069e-02 -4.47578393e-02 -7.97315761e-02 -1.80238821e-02
  1.65816829e-01  3.74060690e-01  4.61086817e-02  6.00010864e-02
 -1.17741935e-01 -7.76912197e-02 -2.82375310e-02 -1.13792568e-01
 -2.93733720e-02  5.69721535e-02  2.53809225e-02  1.08619863e-02
  5.20594679e-02 -6.57318346e-03  9.95274186e-02 -1.51301891e-01
 -5.19979559e-02 -2.85977349e-02  3.99339199e-02 -8.51704627e-02
 -2.39323429e-03  4.83661629e-02 -5.15437946e-02 -1.06330663e-02
  8.36717039e-02  6.82203993e-02  3.81557643e-02  2.58158278e-02
  4.78198193e-02 -9.13953558e-02  5.77312782e-02  4.75437790e-02
  7.45898709e-02 -4.86135436e-

In [44]:
# Now let's try exporting our vectors and using the sci kit logistic regression model!
# For the sake of time, I will assume we only care about the models trained on clean data and vector size 100. 
import export_wordvecs_fasttext as ev

'''
My dear project partners: here is all the info you need to know about using the fasttext embeddings with sci kit learn.
Checkout export_wordvecs_fasttext.py if you want to see how it works in more detail.

You can do the log reg here if you'd like!

Again, I am not very familiar with the scikit module, but here is some pseudo-code I found on stackexchange that may
work for importing vectors.

  pseudo-code that may work:   
      import export_wordvecs_fasttext as ev

      labels, reviews, vecs = ev.get_vectors(model)    
      
      # Fix up the input

      def transform(x):
          return word_vecs_matrix[x]
      transformer = FunctionTransformer(transform)

      # the transformer can be passed into a logistic regression with Pipeline
      # (I am fuzzy on the details of this)
      pip = Pipeline([(Countvectorizer()), (TfidfTransformer()), (Classifier())])
      pip.fit(X_train, y_train)
'''
# retrieve the embeddings for mc2 -- fasttext model trained on cleaned data with dim=100
ev.get_vectors(mc2)


# maybe here we can try out some vector addition/subtraction to see what kind of meaning the model captures. 

31,  0.12682113, -0.04229665,  0.02095987, -0.08952472,
          0.03076454, -0.00333742, -0.04602464,  0.00854672,  0.06770395,
         -0.07138555,  0.21593243,  0.00151844,  0.02396381,  0.1913036 ,
         -0.18843484,  0.14178748, -0.00722332,  0.04011595,  0.07424541,
         -0.2505626 , -0.02013756, -0.02593511, -0.15387928, -0.13259879,
         -0.0477282 ,  0.07027178,  0.07647592,  0.25728917,  0.11458078,
         -0.00036349,  0.2152779 ,  0.18856826,  0.08416598, -0.17971514,
          0.09330187,  0.09929543, -0.0496739 , -0.05019379,  0.03061431,
         -0.01021076,  0.12061864,  0.11578641,  0.04980656, -0.16913776,
          0.06507883,  0.2390815 ,  0.12097382,  0.2794371 , -0.21470448,
         -0.01765842,  0.05202216,  0.10172028, -0.00946029,  0.2519265 ,
          0.07098101,  0.03566222,  0.17518657, -0.02566543, -0.2552624 ,
         -0.05422689, -0.05580621,  0.25826296,  0.03736095, -0.18397702,
         -0.13760857, -0.1561255 ,  0.06918055, -0.06047